In [29]:
# Importar librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import os
import shutil

In [30]:
# Ruta relativa del dataset local
train_dir = './Wonders of World/train' 
val_dir = './Wonders of World/val'
test_dir = './Wonders of World/test'

In [31]:
# Verifica si las rutas existen
if os.path.exists(train_dir) and os.path.exists(val_dir) and os.path.exists(test_dir):
    print('Data source import complete.')
else:
    print('Error: Una o más rutas especificadas no existen.')

Data source import complete.


In [32]:
# Establecer semilla fija para reproducibilidad
seed = 1728676400
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [33]:
# Generadores de datos con ImageDataGenerator
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.15  # División de validación
)

datagen_test = ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=seed
)

val_generator = datagen_train.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=seed
)

test_generator = datagen_test.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    seed=seed
)

Found 2305 images belonging to 12 classes.
Found 401 images belonging to 12 classes.
Found 570 images belonging to 12 classes.


In [34]:
# Cargar modelo preentrenado Xception
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [35]:
# Crear el modelo
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │        12,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,971,956 (87.63 MB)

 Trainable params: 2,110,476 (8.05 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [36]:
# Entrenar el modelo
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    verbose=2
)

c:\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
73/73 - 285s - 4s/step - accuracy: 0.6703 - loss: 1.0103 - val_accuracy: 0.8404 - val_loss: 0.5583
Epoch 2/30
73/73 - 276s - 4s/step - accuracy: 0.8299 - loss: 0.5459 - val_accuracy: 0.8379 - val_loss: 0.5078
Epoch 3/30
73/73 - 276s - 4s/step - accuracy: 0.8564 - loss: 0.4462 - val_accuracy: 0.8753 - val_loss: 0.4141
Epoch 4/30
73/73 - 276s - 4s/step - accuracy: 0.8529 - loss: 0.4252 - val_accuracy: 0.8753 - val_loss: 0.3701
Epoch 5/30
73/73 - 274s - 4s/step - accuracy: 0.8785 - loss: 0.3679 - val_accuracy: 0.8479 - val_loss: 0.4691
Epoch 6/30
73/73 - 276s - 4s/step - accuracy: 0.8937 - loss: 0.2965 - val_accuracy: 0.8853 - val_loss: 0.3869
Epoch 7/30
73/73 - 271s - 4s/step - accuracy: 0.8742 - loss: 0.3988 - val_accuracy: 0.8728 - val_loss: 0.4407
Epoch 8/30
73/73 - 271s - 4s/step - accuracy: 0.8976 - loss: 0.3090 - val_accuracy: 0.8579 - val_loss: 0.4237
Epoch 9/30
73/73 - 271s - 4s/step - accuracy: 0.9046 - loss: 0.2807 - val_accuracy: 0.8404 - val_loss: 0.4988
Epoch 10/3

In [37]:
# Evaluar el modelo
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test accuracy:', test_acc)

18/18 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.9081 - loss: 0.2955
Test accuracy: 0.9087719321250916


In [38]:
# Guardar el modelo en formato .keras
model.save('./modelNella.keras')

In [39]:
# Convertir el modelo a TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('./modelNella.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpk0jew9__\assets


INFO:tensorflow:Assets written to: C:\Users\ADMINI~1\AppData\Local\Temp\tmpk0jew9__\assets


Saved artifact at 'C:\Users\ADMINI~1\AppData\Local\Temp\tmpk0jew9__'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_270')
Output Type:
  TensorSpec(shape=(None, 12), dtype=tf.float32, name=None)
Captures:
  2033424880400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424890192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424890000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424891152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424890960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424889616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424888272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424888080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424888464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2033424889232: TensorSpec(shape=(), dtype=tf.resource, name=None)